# Trainen van een simpel TensorFlow Lite model voor microcontrollers

Deze notebook demonstreert het proces voor het trainen van een `2.5 kB` model met TensorFlow en het converteren voor gebruik met TensorFlow Lite voor microcontrollers.

Deep learning netwerken leren patronen in onderliggende data te modelleren. Hier gaan we een netwerk trainen om gegevens te modelleren die zijn gegenereerd door een sinus functie. Dit zal resulteren in een model dat een waarde, `x`, kan aannemen en zijn sinus, `y`, kan voorspellen.

Deze notebook is gebaseerd op de originele [hello world demo van TensorFlow Lite](https://github.com/tensorflow/tflite-micro/blob/main/tensorflow/lite/micro/examples/hello_world/train/train_hello_world_model.ipynb).

## In een virtuele environment

Je kan deze notebook ook runnen op een computer in een virtuele python environment aan de hand van `conda`.

```bash
conda create -n ts-flow python=3.9
conda activate ts-flow
```

Vergeet in VSCode dan niet om je kernel correct te zetten.

## Configure Defaults

In [ ]:
# Define paths to model files
import os
MODELS_DIR = 'models/'
if not os.path.exists(MODELS_DIR):
    os.mkdir(MODELS_DIR)
MODEL_TF = MODELS_DIR + 'model'
MODEL_NO_QUANT_TFLITE = MODELS_DIR + 'model_no_quant.tflite'
MODEL_TFLITE = MODELS_DIR + 'model.tflite'
MODEL_TFLITE_MICRO = MODELS_DIR + 'model.cc'

## Opzetten van omgeving

Installeren van de nodige libraries. Hier in commentaar gezet omdat dit reeds voor jullie is gedaan. Dit duurt ook vrij lang.


In [ ]:
# ! pip install tensorflow==2.4.0
# ! pip install pandas
# ! pip install matplotlib

Importeren van de nodige libraries.
- **tensorflow:** open source machine learning library
- **keras:** high-level API rond tensorflow voor deep learning
- **numpy:** wiskundige library
- **panda:** data manipulatie library
- **matplotlib:** library voor grafieken
- **math:** wiskundige library

In [ ]:
# TensorFlow is an open source machine learning library
import tensorflow as tf

# Keras is TensorFlow's high-level API for deep learning
from tensorflow import keras
# Numpy is a math library
import numpy as np
# Pandas is a data manipulation library 
import pandas as pd
# Matplotlib is a graphing library
import matplotlib.pyplot as plt
# Math is Python's math library
import math

# Set seed for experiment reproducibility
seed = 1
np.random.seed(seed)
tf.random.set_seed(seed)

## Maken van een simpele dataset

In deze stap gaan we een artificiële dataset generen. In de realiteit zouden we dit bv. kunnen opmeten met een automatische meetopstelling.

In de praktijk is het model maar zo goed als de originele dataset waarmee we het trainen. Vandaar dat het in de praktijk heel belangrijk is om een goede dataset te hebben.

### 1. Genereren van de data

In onderstaande code genereren we een random set van `x` waarden. Voor elke `x` waarde berekenen we dan de `sinus(x)`, welke dan onze `y` waarden worden.

Het resultaat geven we terug in een grafiek.

In [ ]:
# Number of sample datapoints
SAMPLES = 1000

# Generate a uniformly distributed set of random numbers in the range from
# 0 to 2π, which covers a complete sine wave oscillation
x_values = np.random.uniform(
    low=0, high=2*math.pi, size=SAMPLES).astype(np.float32)

# Shuffle the values to guarantee they're not in order
np.random.shuffle(x_values)

# Calculate the corresponding sine values
y_values = np.sin(x_values).astype(np.float32)

# Plot our data. The 'b.' argument tells the library to print blue dots.
plt.plot(x_values, y_values, 'b.')
plt.show()

### 2. Toevoegen van ruis

Onze data werd rechtstreeks gegeneerd door de sinus functie en heeft daarom een mooie vorm.

Machine learning modellen zijn echter goed in het extraheren van de onderliggende betekenis van rommelige data uit de echte wereld. Om dit aan te tonen, kunnen we wat ruis aan onze data toevoegen om een iets realistischere situatie te benaderen.

Onderstaande code voegt wat willekeurige ruis toe aan elke waarde. Het resultaat wordt opnieuw in een grafiek weergegeven.

In [ ]:
# Add a small random number to each y value
y_values += 0.1 * np.random.randn(*y_values.shape)

# Plot our data
plt.plot(x_values, y_values, 'b.')
plt.show()

### 3. Splits de data

De dataset die we nu hebben gemaakt is een benadering van een realistische dataset. Deze waarden gaan we nu gebruiken om ons model te trainen.

Om de nauwkeurigheid van het model dat we trainen te evalueren, moeten we de voorspellingen vergelijken met echte gegevens en controleren hoe goed ze overeenkomen. Deze evaluatie vindt plaats tijdens de training (waar dit **validatie** wordt genoemd) en na de training (ook wel **testen** genoemd). In beide gevallen is het belangrijk dat we nieuwe gegevens gebruiken die nog niet zijn gebruikt om het model te trainen.

We dienen onze dataset dus op te splitsen in 3 groepen:

1. **Trainingsdata:** 60% (het grootste aantal gebruiken we om het model te trainen)
2. **Validatie:** 20% (wordt gebruikt tijdens het trainen om model bij te sturen)
3. **Testen:** 20% (wordt gebruikt na het trainen om te kijken hoe goed het model werkt)

Onderstaande code zal onze originele dataset opsplitsen in deze verschillende sets en grafisch weergeven met elk hun eigen kleur.

In [ ]:
# We'll use 60% of our data for training and 20% for testing. The remaining 20%
# will be used for validation. Calculate the indices of each section.
TRAIN_SPLIT =  int(0.6 * SAMPLES)
TEST_SPLIT = int(0.2 * SAMPLES + TRAIN_SPLIT)

# Use np.split to chop our data into three parts.
# The second argument to np.split is an array of indices where the data will be
# split. We provide two indices, so the data will be divided into three chunks.
x_train, x_test, x_validate = np.split(x_values, [TRAIN_SPLIT, TEST_SPLIT])
y_train, y_test, y_validate = np.split(y_values, [TRAIN_SPLIT, TEST_SPLIT])

# Double check that our splits add up correctly
assert (x_train.size + x_validate.size + x_test.size) ==  SAMPLES

# Plot the data in each partition in different colors:
plt.plot(x_train, y_train, 'b.', label="Training")
plt.plot(x_test, y_test, 'r.', label="Testing")
plt.plot(x_validate, y_validate, 'y.', label="Validation")
plt.legend()
plt.show()


## Training

Nu onze dataset klaar is, kunnen we starten met het maken van een neuraal netwerk om het vervolgens te trainen met de data die we uit vorige stap hebben.

### 1. Creëren van een Model

In deze stap gaan we een simpel neural netwerk model bouwen dat een input waarde (hier in ons geval de `x` parameter) neemt en dit gebruikt om de bijhorende sinus waarde te voorspellen (hier voorgesteld door `y`).

In dit geval hebben we te maken met **regressie**. Regressie is een statistische methode die in AI wordt gebruikt om de relatie tussen een afhankelijke variabele en een of meer onafhankelijke variabelen te modelleren. Het wordt gebruikt voor het voorspellen van continue waarden (reële getallen) voor een bepaalde invoer. Het model wordt vervolgens gebruikt om voorspellingen te doen op nieuwe, ongeziene gegevens.

TODO: Is onderstaande correct?

Het neuraal netwerk zal bestaan uit volgende onderdelen:

1. Een input die onze `x` waarde zal bevatten
2. Een **hidden layer** met 8 **neuronen**
    - Op basis van deze input zal elk neuron _geactiveerd_ worden met een bepaalde waarde en dit op basis van zijn interne parameters (_gewicht_ (_weight_) en _offset_ (_bias_)). De activeringsgraad van een neuron wordt uitgedrukt als een getal.
3. Een **output** layer die het resultaat van de 8 neuronen zal samennemen om een voorspelling te maken van de `y` waarde
    - De activeringsnummers van onze verborgen layer zullen als invoer worden gebruikt voor onze output layer, die een enkele neuron is. Het zal zijn eigen gewichten en offset toepassen op deze invoer en zijn eigen activering berekenen, die zal resulteren in een `y`-waarde.

<!-- De input is hier niet echt een layer maar eerder een input object. Meer info hier: [https://www.tensorflow.org/guide/keras/sequential_model](https://www.tensorflow.org/guide/keras/sequential_model). -->

![Neural Network](./img/neural_network.jpg)

TODO: Update drawing above

De code hieronder maakt een model zoals we hierboven hebben beschreven. Eens dit samengesteld is compileren we dit zodat het klaar is om te trainen.

Als output zie je een klein overzicht van je gemaakt model.

In [ ]:
# We'll use Keras to create a simple model architecture
model_1 = tf.keras.Sequential(name="basic-model")

# First layer takes a scalar input and feeds it through 8 "neurons". The
# neurons decide whether to activate based on the 'relu' activation function.
model_1.add(keras.layers.Dense(8, activation='relu', input_shape=(1,), name="hidden-layer"))

# Final layer is a single neuron, since we want to output a single value
model_1.add(keras.layers.Dense(1, name="output-layer"))

# Compile the model using the standard 'adam' optimizer and the mean squared error or 'mse' loss function for regression.
model_1.compile(optimizer='adam', loss='mse', metrics=['mae'])

# Output some information about the model
model_1.summary()

### 2. Trainen van het Model

Zodra we het model hebben gedefinieerd, kunnen we onze gegevens gebruiken om het te **trainen**. Training bestaat uit aanvoeren van een `x`-waarde aan het neurale netwerk, controleren hoe ver de output van het netwerk afwijkt van de verwachte `y`-waarde en het aanpassen van de gewichten en offsets van de neuronen, zodat de kans groter is dat de output de volgende keer correct is.

Training voert dit proces meerdere keren uit op de volledige dataset en elke volledige doorloop wordt een **epoch** genoemd. Het aantal epochs dat tijdens de training moet worden uitgevoerd, is een parameter die we kunnen instellen.

Tijdens elke epoch, wordt een deel van de dataset door het netwerk gestuurd. Deze subset van de originele training set noemen we een **batch**. Elke batch wordt door het netwerk gestuurd en de bijhorende output van elke input wordt bijgehouden. De correctheid van de outputs wordt vervolgens opgemeten en geaggregeerd. Op basis hiervan worden dan de gewichten en offset van het netwerk aangepast.

TODO: Figuur van trainingsproces?

De grootte van de batch kan naar voorkeur ook worden aangepast.

Onderstaande code gebruikt de waarden `x` en `y` uit onze training set om het model te trainen. Het draait voor 500 _epochs_, met 64 stukjes data in elke _batch_. We geven ook enkele gegevens door voor _validatie_. Zoals je zult zien als je code uitvoert is dit een vrij intensief proces. Het trainen van een AI model vraagt heel wat rekenkracht, en dit is een super simpel model.

In [ ]:
# Train the model on our training data while validating on our validation set
history_1 = model_1.fit(x_train, y_train, epochs=500, batch_size=64,
                        validation_data=(x_validate, y_validate))

### 3. Analyse van de statistieken

In dit deel gaan we dieper in op de prestaties van ons getrained model.

**1. Loss (of gemiddelde kwadratische fout)**
<!-- Loss (or Mean Squared Error) -->

Tijdens het trainen, wordt de prestatie van het model constant gemeten op basis van de trainings data en de validatie data (aparte data set). Het trainingsproces voorziet een volledige historiek van de prestaties van het model tijdens het trainen zodat we de evolutie van het trainingsproces achteraf kunnen analyseren.

In volgende secties gaan we hier dieper op in.

In [ ]:
# Draw a graph of the loss, which is the distance between
# the predicted and actual values during training and validation.
train_loss = history_1.history['loss']
val_loss = history_1.history['val_loss']

epochs = range(1, len(train_loss) + 1)

plt.plot(epochs, train_loss, 'g.', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

De grafiek toont de **loss** (verlies). Dit is het verschil tussen de door het model voorspelde waarde en de eigenlijk data en dit voor elke epoch. De `loss`-waarde wordt hierbij per epoch bepaald door de gemiddelde kwadratische fout te berekenen (er zijn ook nog andere manieren).

Merk op dat er een `loss` wordt getoond voor zowel de originele training set als voor de validatie set.

We zien in de grafiek dat de `loss` snel afneemt over de eerste 25 epochs, waarna deze begint te stabiliseren. Dit betekent dat het model in de eerste 25 epochs sterk verbetert en nauwkeurigere voorspellingen produceert.

Ons doel is om te stoppen met trainen wanneer het niet meer verbeterd of wanneer de `training loss` lager wordt dan de `validation loss`, in welk geval het model geleerd heeft om de trainings data zo goed te voorspellen dat het niet meer beter kan met nieuwe data.

Om te kijken waar we best stoppen met trainen, kunnen we best even inzoomen op de grafiek op het gedeelte **na de eerste 50 epochs**.

In [ ]:
# Exclude the first few epochs so the graph is easier to read
SKIP = 50

plt.plot(epochs[SKIP:], train_loss[SKIP:], 'g.', label='Training loss')
plt.plot(epochs[SKIP:], val_loss[SKIP:], 'b.', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

Uit de grafiek kunnen we concluderen dat de `loss` daalt tot ongeveer 200 epochs. Vanaf daar begint het te stabiliseren. Dit betekent dat we eigenlijk **na 200 epochs kunnen stoppen** met het trainen van ons model.

We kunnen echter ook zien dat de laagste `loss` rond de `0.155` ligt. Dit betekent dat de voorspellingen van ons netwerk gemiddeld `~15%` afwijken. Bovendien springen de `validation loss` waarden veel rond en zijn ze soms zelfs hoger.

TODO: We kunnen echter ook zien dat de laagste `loss` rond de `0.145` ligt. Dit betekent dat de voorspellingen van ons netwerk gemiddeld `~14%` afwijken. Daarnaast zien we dat de `validation loss` waarden een stuk hoger liggen, namelijk rond de `0.165`.

**2. Gemiddelde absolute fout**
<!-- Mean Absolute Error -->

Om meer inzicht te krijgen in de prestaties van ons model kunnen we wat meer gegevens in een grafiek plaatsen. Deze keer zullen we de gemiddelde absolute fout analyseren, wat een andere manier is om te meten hoe ver de voorspellingen van het netwerk verwijderd zijn van de werkelijke waarden.

In [ ]:
plt.clf()

# Draw a graph of mean absolute error, which is another way of
# measuring the amount of error in the prediction.
train_mae = history_1.history['mae']
val_mae = history_1.history['val_mae']

plt.plot(epochs[SKIP:], train_mae[SKIP:], 'g.', label='Training MAE')
plt.plot(epochs[SKIP:], val_mae[SKIP:], 'b.', label='Validation MAE')
plt.title('Training and validation mean absolute error')
plt.xlabel('Epochs')
plt.ylabel('MAE')
plt.legend()
plt.show()

Deze grafiek van _gemiddelde absolute fout_ vertelt een ander verhaal. We kunnen zien dat trainings gegevens consistent minder fouten vertonen dan validatie gegevens, wat betekent dat het netwerk mogelijks **overfit** is, of de trainingsgegevens zo goed heeft geleerd dat het geen effectieve voorspellingen kan doen over nieuwe gegevens.

Bovendien zijn de `MAE` waarden behoorlijk hoog, op zijn best `~0,305`, wat betekent dat sommige voorspellingen van het model er minstens `30%` naast zitten. Een fout van `30%` betekent dat we nog ver verwijderd zijn van het nauwkeurig modelleren van de sinusgolf functie.

#### Overfitting

Overfitting in een neuraal netwerk verwijst naar een situatie waarin het model de trainingsgegevens te goed heeft geleerd en te complex is geworden, zodat het begint te passen bij de ruis in de gegevens in plaats van bij het onderliggende patroon. Dit resulteert in slechte generalisatieprestaties, wat betekent dat het model goed presteert op de trainingsgegevens, maar slecht op nieuwe, ongeziene gegevens.

<!--
MSE vs MAE
In summary, MSE and MAE are both measures of the difference between the predictions and actual values, with MSE emphasizing larger errors and MAE providing a more robust measurement. The choice of loss function depends on the nature of the problem and the desired trade-off between sensitivity to larger errors and robustness to outliers.
-->

#### 3. Echte waarden versus voorspelde waarden

Laten we, om meer inzicht te krijgen in wat er gebeurt, de voorspellingen vergelijken met de testdataset die we eerder opzij hebben gezet:

In [ ]:
# Calculate and print the loss on our test dataset
test_loss, test_mae = model_1.evaluate(x_test, y_test)

# Make predictions based on our test dataset
y_test_pred = model_1.predict(x_test)

# Graph the predictions against the actual values
plt.clf()
plt.title('Comparison of predictions and actual values')
plt.plot(x_test, y_test, 'b.', label='Actual values')
plt.plot(x_test, y_test_pred, 'r.', label='TF predictions')
plt.legend()
plt.show()

Slecht nieuws. De grafiek maakt duidelijk dat ons netwerk geleerd heeft om de sinusfunctie zeer beperkt te benaderen.

De rigiditeit van deze fit suggereert dat het model niet genoeg capaciteit heeft om de volledige complexiteit van de sinusgolf functie te leren. Door ons model groter te maken, zouden we de prestaties ervan moeten kunnen verbeteren.

Merk op dat dit net het omgekeerde is van een overfit, waar het model te complex is. Nu aangezien we maar 1 hidden layer hebben, is de kans wel heel klein dat we met een overfit zitten.

## Trainen van een groter model

Open de 2de notebook `larger_model.ipynb` om verder te gaan.